# Search White House Speeches from 2021 to 2022 Based On Content

A semantic search example based on White House Speeches from 2021 to 2022. Many of these speeches were made after GPT 3.5 was trained. The White House (Speeches and Remarks) 12/10/2022 dataset can be found on [Kaggle](https://www.kaggle.com/datasets/mohamedkhaledelsafty/the-white-house-speeches-and-remarks-12102022). For this example, we've also made this available on Google Drive. We put together a system to semantically search these speeches using a vector database and the sentence-transformers library. For this example, we use [Milvus Lite](https://milvus.io/docs/milvus_lite.md) to run our vector database locally.

We begin by installing the necessary libraries:

In [ ]:
! pip install pymilvus sentence-transformers gdown milvus

## Download Dataset

Next, we download and extract our dataset

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=10_sVL0UmEog7mczLedK5s1pnlDOz3Ukf'
output = './white_house_2021_2022.zip'
gdown.download(url, output)

import zipfile

with zipfile.ZipFile("./white_house_2021_2022.zip","r") as zip_ref:
    zip_ref.extractall("./white_house_2021_2022")

## Clean the Data

This dataset is not a precleaned dataset so we need to clean it up before we can work on it. Our first preprocessing step is to drop all rows with any `Null` or `NaN` data using `.dropna()`. Next, we ensure that we aren't picking up any partial speeches by only taking speeches that have more than 50 characters. We also get rid of all the return and newline characters in the speeches. Finally, we convert the dates into the universally accepted datetime format.

In [ ]:
import pandas as pd
df = pd.read_csv("./white_house_2021_2022/The white house speeches.csv")
df.head()

In [ ]:
df = df.dropna()

In [ ]:
cleaned_df = df.loc[(df["Speech"].str.len() > 50)]

In [ ]:
cleaned_df

In [ ]:
cleaned_df["Speech"] = cleaned_df["Speech"].str.replace("\r\n", "")
cleaned_df.iloc[0]["Speech"]

In [ ]:
import datetime

# Convert the 'date' column to datetime objects
cleaned_df["Date_time"] = pd.to_datetime(cleaned_df["Date_time"], format="%B %d, %Y")

# Convert the datetime objects to Unix time format
cleaned_df["unix_time"] = cleaned_df["Date_time"].apply(lambda x: int(x.timestamp()))

cleaned_df

## Establish a Vector Database and Schema

With all of our datacleaning done, it's time to set up our vector database, Milvus Lite. We start by declaring some constants before starting a server and establishing a connection.

In [ ]:
COLLECTION_NAME = "white_house_2021_2022"
DIMENSION = 384
BATCH_SIZE = 128
TOPK = 3

In [ ]:
from milvus import default_server
from pymilvus import connections, utility

default_server.start()
connections.connect(host="127.0.0.1", port=default_server.listen_port)

utility.get_server_version()

Just to make sure that we are starting from a blank slate, we check for the existence of any collection with the same name as the one we chose and drop it.

In [ ]:
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

Now we establish our schema. For this data set, we have four attributes to work off - the title of the speech, the date the speech was given, the location where the speech was given, and the speech itself. We want to perform a semantic search on the content of the actual speech so the schema will contain the title, the date, the location, and a vector embedding of the actual speech.

For each `VARCHAR` datatype (string format) we give a max length. In this case, none of these max lengths are hit, but serve as a rough upper bound estimate.

In [ ]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

# object should be inserted in the format of (title, date, location, speech embedding)
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="date", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="location", dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)

With a vector database server up and running as well as a collection and schema established, the final thing to do before inserting the vectors is to establish our vector index. For this example, we use an `IVF_FLAT` index on an `L2` distance metric and 128 clusters (`nlist`).

In [ ]:
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
collection.create_index(field_name="embedding", index_params=index_params)
collection.load()

## Get Vector Embeddings and Populate the Database 

Here we use the `SentenceTransformer` library to get our vector embeddings for the speeches and populate our Milvus instance with our newly generated vector embeddings. For this example, we use the [MiniLM L6 v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) transformer to get a vector embedding.

In [ ]:
from sentence_transformers import SentenceTransformer

transformer = SentenceTransformer('all-MiniLM-L6-v2')

We create a function, `embed_insert`, that gets the embeddings for a batch of speeches, and then inserts that batch into our Milvus instance.

In [ ]:
# expects a tuple of (title, date, location, speech)
def embed_insert(data: list):
    embeddings = transformer.encode(data[3])
    ins = [
        data[0],
        data[1],
        data[2],
        [x for x in embeddings]
    ]
    collection.insert(ins)

With our helper function written, we are ready to embed and insert the text. First, we turn our `pandas` dataframe into the right format, a list of lists, to insert. For this example, we need a list of four lists. The inner lists correspond to the title, date, location, and speech respectively. We batch the lists and call the `embed_insert` function we wrote above on each of them. Finally, when all of the data has been inserted, we `flush` the collection to ensure that everything is indexed.

In [ ]:
data_batch = [[], [], [], []]

for index, row in cleaned_df.iterrows():
    data_batch[0].append(row["Title"])
    data_batch[1].append(str(row["Date_time"]))
    data_batch[2].append(row["Location"])
    data_batch[3].append(row["Speech"])
    if len(data_batch[0]) % BATCH_SIZE == 0:
        embed_insert(data_batch)
        data_batch = [[], [], [], []]

# Embed and insert the remainder
if len(data_batch[0]) != 0:
    embed_insert(data_batch)

# Call a flush to index any unsealed segments.
collection.flush()

## Run a Semantic Search

With the database populated, it's now possible to search all of the speeches based on their content. In this example, we search for a speech where the President speaks about renewable energy at NREL, and a speech where the Vice President and the Prime Minister of Canada both speak. We get the embeddings for these descriptions, and then search our vector database for the 3 speeches with the closest embeddings. 

We expect the first description to have the speech titled "Remarks by President Biden During a Tour of the National Renewable Energy Laboratory" in its results and the second description to have the speech titled "REMARKS BY VICE PRESIDENT HARRIS AND PRIME MINISTER TRUDEAU OF CANADA BEFORE BILATERAL MEETING" in its results.

In [ ]:
import time
search_terms = ["The President speaks about the impact of renewable energy at the National Renewable Energy Lab.", "The Vice President and the Prime Minister of Canada both speak."]

# Search the database based on input text
def embed_search(data):
    embeds = transformer.encode(data) 
    return [x for x in embeds]

search_data = embed_search(search_terms)

start = time.time()
res = collection.search(
    data=search_data,  # Embeded search value
    anns_field="embedding",  # Search across embeddings
    param={"metric_type": "L2",
            "params": {"nprobe": 10}},
    limit = TOPK,  # Limit to top_k results per search
    output_fields=["title"]  # Include title field in result
)
end = time.time()

for hits_i, hits in enumerate(res):
    print("Title:", search_terms[hits_i])
    print("Search Time:", end-start)
    print("Results:")
    for hit in hits:
        print( hit.entity.get("title"), "----", hit.distance)
    print()

Clean up the server.

In [ ]:
default_server.stop()